<a href="https://colab.research.google.com/github/dtminnick/pytorch-practice/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook completes the requirements for Deep Learning Module 2

Import libraries.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

Load the MNIST data.

In [4]:
# Convert MNIST image files to tensors of four dimensions.
# Number of images, height, width, color channel.

transform = transforms.ToTensor()

# Get training data.

train_data = datasets.MNIST(root = "/cnn_data",
                            train = True,
                            download = True,
                            transform = transform)

# Get test data.

test_data = datasets.MNIST(root = "/cnn_data",
                           train = False,
                           download = True,
                           transform = transform)

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

Confirm download of training data.

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

Confirm download of test data.

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()